# Turtle runs - RL with the smoothed Crossentropy Method

In [89]:
import sys
import numpy as np
from collections import namedtuple
from time import time

In [90]:
actions = ['N', 'E', 'S', 'W', 'O']

In [91]:
class Turtle():
    def __init__(self, position, halite):
        self.position = position
        self.halite = halite

In [159]:
class GameState():
    def __init__(self, game_map, position, halite):
        self.game_map = game_map
        self.position = position
        self.halite = halite
        
    def __eq__(self, other):
        return (self.game_map == other.game_map).all() and self.position == other.position and self.halite == other.halite

    def __contains__(self, key):
        return key in self.numbers

    def get_nn_repr(self):
        hal_std_scaled = (game_map.reshape(-1,) - 500) / 1000
        pos_indicator = [0] * 25
        pos_indicator[self.position[0] * 5 + self.position[1]] = 1
        return list(hal_std_scaled) + pos_indicator

In [131]:
class SimpleHalite():
    def __init__(self, height, width, start_pos):
        np.random.seed(42)
        self.game_map = np.random.randint(1, 1000, size=(height, width))
        self.game_map[start_pos] = 0
        self.orig_map = self.game_map.copy()
        self.turtle = Turtle(start_pos, 0)
        self.turn = 1
        self.max_turns = 50
        self.halite = 0
        self.base = start_pos
        self.height = height
        self.width = width
        self.actions = ['N', 'E', 'S', 'W', 'O']
    
    def get_state(self):
        game_state = GameState(self.game_map.copy(), self.turtle.position, self.turtle.halite)
        return game_state
    
    def get_possible_actions(self):
        return self.actions
    
    def reset(self):
        self.game_map = self.orig_map.copy()
        self.turtle = Turtle(self.base, 0)
        self.turn = 1
        self.halite = 0
        return self.get_state()
        
    def step(self, action):
        reward = 0
        if action == 'O':
            mined_halite = self.game_map[self.turtle.position] // 4
            self.game_map[self.turtle.position] -= mined_halite
            self.turtle.halite += min(1000, mined_halite)
        else:
            if action == 'N':
                cost_halite = self.game_map[self.turtle.position] // 10
                new_pos = tuple([sum(x) for x in zip(self.turtle.position, (0, 1))])
            elif action == 'E':
                cost_halite = self.game_map[self.turtle.position] // 10
                new_pos = tuple([sum(x) for x in zip(self.turtle.position, (1, 0))])
            elif action == 'S':
                cost_halite = self.game_map[self.turtle.position] // 10
                new_pos = tuple([sum(x) for x in zip(self.turtle.position, (0, -1))])
            elif action == 'W':
                cost_halite = self.game_map[self.turtle.position] // 10
                new_pos = tuple([sum(x) for x in zip(self.turtle.position, (-1, 0))])
            #print(cost_halite, self.turtle.halite)
            if cost_halite <= self.turtle.halite:
                #print("moving turtle to {}".format(new_pos))
                self.turtle = Turtle(new_pos, self.turtle.halite - cost_halite)
            else:
                mined_halite = self.game_map[self.turtle.position] // 4
                self.game_map[self.turtle.position] -= mined_halite
                self.turtle.halite += min(1000, mined_halite)                
        self.turtle.position = (self.turtle.position[0] % self.width, 
                                self.turtle.position[1] % self.height)
        if self.turtle.position == self.base:
            self.halite += self.turtle.halite
            reward = self.turtle.halite
            self.turtle.halite = 0
        self.turn += 1
        return self.get_state(), reward, self.turn == self.max_turns

In [132]:
def policy(game_state, turn, policy_mapping=None):
    if policy_mapping is not None and (game_state.position, game_state.halite) in policy_mapping:
        for map_action in policy_mapping[(game_state.position, game_state.halite)]:
            if (map_action[0] == game_state.game_map).all():
                return np.random.choice(actions, p=map_action[1])
    return np.random.choice(actions)

Needs to be a tuple rather than a list as start_pos

In [133]:
game = SimpleHalite(5, 5, (2, 2))

In [134]:
game.reset()

In [144]:
N = 200
M = 10
alpha = 0.5

In [136]:
def add_to_policy_mapping(policy_mapping, game_map, position, halite, action_probs):
    if (position, halite) not in policy_mapping:
            policy_mapping[(position, halite)] = []
    policy_mapping[(position, halite)].append((game_map, action_probs))
    return

In [137]:
import warnings
warnings.filterwarnings("ignore")

In [138]:
class ArrayWrapper():
    def __init__(self):
        self.els = []
    
    def append(self, obj):
        self.els.append(obj)
        
    def __contains__(self, obj):
        for el in self.els:
            if (el == obj).all():
                return True
        return False

In [148]:
#policy_mapping = {}
for j in range(M):
    now = time()
    state_action_maps = []
    final_rewards = []
    for i in range(N):
        state_action_map = []
        game_state = game.reset()
        done = False
        total_reward = 0
        while not done:
            action = policy(game_state, turn, policy_mapping)
            state_action_map.append((game_state, action))
            game_state, reward, done = game.step(action)
            total_reward += reward
        state_action_maps.append(state_action_map)
        final_rewards.append(total_reward)
    print(j, np.mean(final_rewards))
    elite_games = np.argsort(final_rewards)[N//5:]
    maps_to_keep = np.array(state_action_maps)[elite_games]
    full_map = {}
    for sam in maps_to_keep:
        for sap in sam:
            if (sap[0].position, sap[0].halite) not in full_map:
                full_map[(sap[0].position, sap[0].halite)] = []
            full_map[(sap[0].position, sap[0].halite)].append((sap[0].game_map, sap[1]))
    print("Entries in policy mapping:", sum(len(x) for x in policy_mapping.values()))
    print("Number of keys:", len(policy_mapping.keys()))
    updated, not_updated = 0, 0
    for ((position, halite), saps) in full_map.items():
        done_policies = ArrayWrapper()
        for (state_map, action) in saps:
            if state_map not in done_policies:
                done_policies.append(state_map)
                subset = [a for (s, a) in saps if (s == state_map).all()]
                action_probs = []
                action_probs_raw = []
                for action in actions:
                    action_probs.append((subset.count(action) + 1) / (len(subset) + 5))
                    action_probs_raw.append(subset.count(action) / len(subset))
                if (position, halite) in policy_mapping: 
                    found = False
                    for ix, (game_map, probs) in enumerate(policy_mapping[position, halite]):
                        if (game_map == state_map).all():
                            policy_mapping[(position, halite)][ix] = \
                                (state_map, alpha * np.array(action_probs_raw) + (1 - alpha) * np.array(probs))
                            updated += 1
                            found = True
                            break
                    if not found:
                        not_updated += 1
                        add_to_policy_mapping(policy_mapping, state_map, position, halite, action_probs)
                else:
                    not_updated += 1
                    add_to_policy_mapping(policy_mapping, state_map, position, halite, action_probs)
    print("Updated: {}, not updated: {}".format(updated, not_updated))
    print("Iteration took {:.0f} seconds".format(time() - now))

0 1444.64
Entries in policy mapping: 175914
Number of keys: 14876
Updated: 82, not updated: 0
Iteration took 49 seconds
1 1453.28
Entries in policy mapping: 175914
Number of keys: 14876
Updated: 49, not updated: 0
Iteration took 49 seconds
2 1454.36
Entries in policy mapping: 175914
Number of keys: 14876
Updated: 49, not updated: 0
Iteration took 49 seconds
3 1460.48
Entries in policy mapping: 175914
Number of keys: 14876
Updated: 49, not updated: 0
Iteration took 50 seconds
4 1461.56
Entries in policy mapping: 175914
Number of keys: 14876
Updated: 49, not updated: 0
Iteration took 50 seconds
5 1462.64
Entries in policy mapping: 175914
Number of keys: 14876
Updated: 49, not updated: 0
Iteration took 50 seconds
6 1462.64
Entries in policy mapping: 175914
Number of keys: 14876
Updated: 49, not updated: 0
Iteration took 50 seconds
7 1462.28
Entries in policy mapping: 175914
Number of keys: 14876
Updated: 49, not updated: 0
Iteration took 44 seconds
8 1463.0
Entries in policy mapping: 1759

We can now simulate the game loop. We now need to 
- fix the game map (otherwise the state mappong will become too unwieldy) - DONE
- keep track of state-action pairs (memory intensive, but maybe we can manage) - DONE
- use state-action pairs from elite runs to update policy - DONE
- add smoothing - DONE
- improve policy mapping data structure to keep turn times roughly constant - DONE

## Approximate crossentropy method 

Use a NN to encode the state

In [160]:
from sklearn.neural_network import MLPClassifier
agent = MLPClassifier(hidden_layer_sizes=(64,32,32),
                      activation='relu',
                      warm_start=True, #keep progress between .fit(...) calls
                      max_iter=2 #make only 1 iteration on each .fit(...)
                     )
n_actions = len(game.get_possible_actions())
#initialize agent to the dimension of state an amount of actions
agent.fit([game.reset().get_nn_repr()]*n_actions, game.get_possible_actions())

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(64, 32, 32), learning_rate='constant',
       learning_rate_init=0.001, max_iter=2, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=True)

To do today:
- Get toy environment working with a NN state space approximation (following 'Going deeper')
- Finish k-armed bandit chapter in Barto Sutton and read at least first half of Chapter 3
- Work on hooking up a "policy" to Halite game engine, as well as collecting state - action - reward pairs during "training" episodes
- Think about radius formalism for heuristics bot - DONE

In [161]:
def train_approx_cem(game, agent, n_iter, n_sessions):
    all_actions = game.get_possible_actions()
    for j in range(n_iter):
        now = time()
        state_action_pairs_list = []
        final_rewards = []
        max_probs = []
        for i in range(n_sessions):
            state_action_pairs = []
            game_state = game.reset()
            done = False
            total_reward = 0
            while not done:
                nn_game_state = game_state.get_nn_repr()
                probs = agent.predict_proba([nn_game_state])[0] 
                max_probs.append(max(probs))
                action = np.random.choice(all_actions, p=probs)
                state_action_pairs.append((nn_game_state, action))
                game_state, reward, done = game.step(action)
                total_reward += reward
            state_action_pairs_list.append(state_action_pairs)
            final_rewards.append(total_reward)
        print(j, np.mean(final_rewards))
        elite_games = np.argsort(final_rewards)[N//5:]
        pairs_to_keep = np.array(state_action_pairs_list)[elite_games]
        elite_states = [sap[0] for sam in pairs_to_keep for sap in sam]
        elite_actions = [sap[1] for sam in pairs_to_keep for sap in sam]
        agent.fit(X=elite_states, y=elite_actions)
        print("Iteration took {:.0f} seconds".format(time() - now))
        print("Mean certainty on best action: {}".format(np.mean(max_probs)))

In [ ]:
train_approx_cem(game, agent, 20, 500)

0 221.534
Iteration took 8 seconds
Mean certainty on best action: 0.2241335992762969
1 208.832
Iteration took 8 seconds
Mean certainty on best action: 0.21986356515222366
2 214.54
Iteration took 7 seconds
Mean certainty on best action: 0.21813028326194436
3 205.282
Iteration took 8 seconds
Mean certainty on best action: 0.21948361788464152
4 220.86
Iteration took 7 seconds
Mean certainty on best action: 0.2162326506051762
5 188.414
Iteration took 8 seconds
Mean certainty on best action: 0.21460489108940067
6 219.712
Iteration took 7 seconds
Mean certainty on best action: 0.21460907436906004
7 229.724
Iteration took 10 seconds
Mean certainty on best action: 0.2195922567180805
8 201.27
Iteration took 8 seconds
Mean certainty on best action: 0.21757810369903938
9 212.48
Iteration took 8 seconds
Mean certainty on best action: 0.21502475412221098
10 216.988
Iteration took 8 seconds
Mean certainty on best action: 0.21775437776020795
11 199.37
Iteration took 8 seconds
Mean certainty on best a

In [158]:
import pickle
f = open(r'./policy.pickle', 'wb')
pickle.dump(policy_mapping, f)
f.close()

In [ ]:
class QLearningAgent:
    def __init__(self, alpha, epsilon, discount, get_legal_actions):
        """
        Q-Learning Agent
        based on http://inst.eecs.berkeley.edu/~cs188/sp09/pacman.html
        Instance variables you have access to
          - self.epsilon (exploration prob)
          - self.alpha (learning rate)
          - self.discount (discount rate aka gamma)

        Functions you should use
          - self.get_legal_actions(state) {state, hashable -> list of actions, each is hashable}
            which returns legal actions for a state
          - self.get_qvalue(state,action)
            which returns Q(state,action)
          - self.set_qvalue(state,action,value)
            which sets Q(state,action) := value

        !!!Important!!!
        Note: please avoid using self._qValues directly. 
            There's a special self.get_qvalue/set_qvalue for that.
        """

        self.get_legal_actions = get_legal_actions
        self._qvalues = defaultdict(lambda: defaultdict(lambda: 0))
        self.alpha = alpha
        self.epsilon = epsilon
        self.discount = discount

    def get_qvalue(self, state, action):
        """ Returns Q(state,action) """
        return self._qvalues[state][action]

    def set_qvalue(self,state,action,value):
        """ Sets the Qvalue for [state,action] to the given value """
        self._qvalues[state][action] = value

    #---------------------START OF YOUR CODE---------------------#

    def get_value(self, state):
        """
        Compute your agent's estimate of V(s) using current q-values
        V(s) = max_over_action Q(state,action) over possible actions.
        Note: please take into account that q-values can be negative.
        """
        possible_actions = self.get_legal_actions(state)

        #If there are no legal actions, return 0.0
        if len(possible_actions) == 0:
            return 0.0

        value = max(self.get_qvalue(state, a) for a in possible_actions)

        return value

    def update(self, state, action, reward, next_state):
        """
        You should do your Q-Value update here:
           Q(s,a) := (1 - alpha) * Q(s,a) + alpha * (r + gamma * V(s'))
        """

        #agent parameters
        gamma = self.discount
        learning_rate = self.alpha

        new_q_val = (1 - learning_rate) * self.get_qvalue(state, action) + learning_rate * (reward + gamma * self.get_value(next_state))
        
        self.set_qvalue(state, action, new_q_val)

    
    def get_best_action(self, state):
        """
        Compute the best action to take in a state (using current q-values). 
        """
        possible_actions = self.get_legal_actions(state)

        #If there are no legal actions, return None
        if len(possible_actions) == 0:
            return None

        best_action = possible_actions[np.argmax([self.get_qvalue(state, a) for a in possible_actions])]

        return best_action

    def get_action(self, state):
        """
        Compute the action to take in the current state, including exploration.  
        With probability self.epsilon, we should take a random action.
            otherwise - the best policy action (self.getPolicy).
        
        Note: To pick randomly from a list, use random.choice(list). 
              To pick True or False with a given probablity, generate uniform number in [0, 1]
              and compare it with your probability
        """

        # Pick Action
        possible_actions = self.get_legal_actions(state)
        action = None

        #If there are no legal actions, return None
        if len(possible_actions) == 0:
            return None

        #agent parameters:
        epsilon = self.epsilon

        if np.random.uniform() < self.epsilon:
            chosen_action = np.random.choice(possible_actions)
        else:
            return self.get_best_action(state)
        
        return chosen_action